# 誤差逆伝播法

## 数値微分の難点

前回までで、ニューラルネットワークの基本的な学習方法である、確率的勾配降下法は実装できた

このとき、勾配の計算は数値微分によって実装していた

数値微分はシンプルに実装できて分かりやすいが、計算に時間がかかるという難点がある

そこで、ここでは、重みパラメータの勾配計算を効率良く行う**誤差逆伝播法**を実装する


## 計算グラフ

誤差逆伝播法を視覚的に理解する方法として**計算グラフ（Computational graph）**というものがある

計算グラフとは計算の過程をグラフによって表したものである

ここで言うグラフとは、データ構造としてのグラフであり、複数のノードとエッジ（ノード間を結ぶ直線）によって表現されるものである

### 計算グラフで計算問題を解く
以下のような簡単な計算問題を、計算グラフを使って解いてみる

**問:** スーパーで1個100円のりんごを2個買ったとき、支払う金額はいくらか？（消費税は10％とする）

計算グラフは以下のように、ノードごとの計算結果が左から右へ伝わるように表現する

![computational_graph01.png](./img/computational_graph01.png)

上記では、「×2」や「×1.1」を一つの演算として○（ノード）でくくっているが、「×」のような演算子は単一のノードで表現するのが望ましい

この場合、「2」と「1.1」は、それぞれ「りんごの個数」と「消費税」という変数として、ノードの外側に表記する

![computational_graph02.png](./img/computational_graph02.png)

次に、もう少し複雑な計算を計算グラフで解く

**問:** スーパーでりんご（100円／個）を2個、みかん（150円／個）を3個買ったとき、支払う金額はいくらか？（消費税は10％とする）

この問題を解くための計算グラフは以下のようになる

![computational_graph03.png](./img/computational_graph03.png)

このように、計算グラフを使って問題を解くには、

1. 計算式に使われる要素を演算子と変数に分ける
2. 演算子と変数をノードとしてエッジでつなぐ
3. 計算グラフ上で計算を左から右へ進める

という流れで作業する

ここで、「計算を左から右へ進める」処理を**順伝播（forward propagation）**と呼び、ニューラルネットワークの**推論処理**に対応する

逆に「計算を右から左へ戻る」処理を**逆伝播（back propagation）**と呼び、ニューラルネットワークの**学習処理**に対応する

この逆伝播は、この先、微分を計算するにあたって重要な働きをする

### 局所的計算と逆伝播による微分
計算グラフを使う利点は大きく以下の2点がある

- 「局所的な計算」を伝播することにより複雑な計算を行うことができる
    - 各ノードは、全体の計算には関与せず「自分に関係する小さな範囲」の計算だけを行う（局所的計算）
    - これにより計算を単純化することができる
    - また、計算途中の結果をすべて、各ノードで保持することも可能
- 計算グラフを逆伝播することで微分値を効率よく計算できる
    - 順伝播が「局所的な計算」であるのと同様に、逆伝播は「局所的な微分」を表す
    - これにより微分計算を単純化し、計算速度を向上させることができる

例えば上記の問題について、りんごの値段が値上がりした場合、最終的な支払金額にどのように影響するか知りたいとする

これは「りんごの値段に対する支払金額の微分」を求めることに相当する（りんごの値段を $x$, 支払金額を $L$ とした場合、$∂L/∂x$ を計算することに相当する）

計算グラフの逆伝播によって、この問題を解くと以下のようになる

![computational_graph04.png](./img/computational_graph04.png)

上記のように、計算グラフを右から左へ計算することで「局所的な微分」を伝達することができる

この結果から「りんごの値段に関する支払金額の微分」の値は 2.2 であると言える

すなわち、りんごが1円値上がりしたら、最終的な支払金額は2.2円増えることを意味する（正確には、りんごの値段がある微小な値だけ増えたら、最終的な支払金額はその微小な値の2.2倍だけ増加することを意味する）

## 連鎖律

### 計算グラフの逆伝播
計算グラフの逆伝播を一般化すると以下のように表すことができる

![computational_graph05.png](./img/computational_graph05.png)

ここで、上記計算グラフは $y = f(x)$ という計算を表現している

逆伝播の計算手順は、信号 $E$ に対して、ノードの局所的な微分 $\frac{∂y}{∂x}$ を乗算し、次のノードで伝達する、というものになっている

これは、前述したりんごの支払金額計算で考えると分かりやすい

```
最初のノード: f(x) = 2x (x: 前のノードの出力値＝りんごの値段, 2: りんごの個数)
最後のノード: g(x) = 1.1x (x: 前のノードの出力値, 1.1: 消費税倍率)

最後のノードの逆伝播: 1 * g'(x) = 1 * 1.1 = 1.1
最初のノードの逆伝播: 1.1 * f'(x) = 1.1 * 2 = 2.2
```

この計算により効率よく微分値を求めることができるだが、その理由は**連鎖律の原理**から説明できる

### 連鎖律
以下のような計算グラフを考える

![computational_graph06.png](./img/computational_graph06.png)

この計算を微分すると、以下のような逆伝播で表現される

![computational_graph07.png](./img/computational_graph07.png)

ここで、合成関数の定理より $\frac{∂z}{∂z} \frac{∂z}{∂y}$ の $∂z$ は "打ち消し合い"、$\frac{∂z}{∂y}$ となる

同様にして $\frac{∂z}{∂z} \frac{∂z}{∂y} \frac{∂y}{∂x} = \frac{∂z}{∂x}$ となる

このような合成関数の微分の性質を**連鎖律**と呼ぶ

連鎖律により、計算の一部を "打ち消す" ことができるため、効率よく微分計算ができるという仕組みである

## 単純な算術ノードの実装

計算グラフの単純なノードを実装していく

ただし、型名は「ノード」ではなく、ニューラルネットワークの「層（レイヤ）」を意味するものとして `***Layer` という名前で実装することにする

In [1]:
# すべてのレイヤの基底となる抽象型: 抽象レイヤ
abstract type AbstractLayer end

### 加算ノード（加算レイヤ）
まず、加算ノード $z = x + y$ について考える

このノードの逆伝播（偏微分）は以下のようになる

$$ \begin{array}{ll}
    \frac{∂z}{∂x} = 1 \\
    \frac{∂z}{∂y} = 1
\end{array} $$

従って、この計算グラフは以下のようになる

![computational_graph_add.png](./img/computational_graph_add.png)

上図のように、加算ノードの逆伝播は、上流の値がそのまま分岐して流れていく

これを実装すると以下のようになる

In [2]:
"""
@note:
    forward, backward は引数のレイヤに対して破壊的な変更を加える可能性がある
    => forward!, backward! という関数名で実装しておく
"""
# 加算レイヤ
mutable struct AddLayer <: AbstractLayer end

# 加算レイヤ: 順伝播
## x, y: 上流から流れてきる2値 => 下流に流す値
forward!(layer::AddLayer, x::Float64, y::Float64)::Float64 = x + y

# 加算レイヤ: 逆伝播
## dout: 上流から流れてくる微分値 => 下流に流す2値
backward!(layer::AddLayer, dout::Float64)::Tuple{Float64,Float64} = (dout * 1, dout * 1)

backward! (generic function with 1 method)

### 乗算ノード（乗算レイヤ）
同様に、乗算ノード $z = x \times y$ について考えると、このノードの逆伝播（偏微分）は以下のようになる

$$ \begin{array}{ll}
    \frac{∂z}{∂x} = y \\
    \frac{∂z}{∂y} = x
\end{array} $$
 
従って、この計算グラフは以下のようになる

![computational_graph_mul.png](./img/computational_graph_mul.png)

すなわち、乗算ノードの逆伝播では、上流から流れてきた微分値に対して、順伝播の "ひっくり返した値" を乗算して流す形になる

これを実装すると以下のようになる

In [3]:
# 乗算レイヤ
mutable struct MulLayer <: AbstractLayer
    x::Float64
    y::Float64
end

MulLayer() = MulLayer(0, 0)

# 乗算レイヤ: 順伝播
## x, y: 上流から流れてきる2値 => 下流に流す値
forward!(layer::MulLayer, x::Float64, y::Float64)::Float64 = begin
    # 順伝播時に値を保持しておく
    layer.x = x
    layer.y = y
    x * y
end

# 乗算レイヤ: 逆伝播
## dout: 上流から流れてくる微分値 => 下流に流す2値
backward!(layer::MulLayer, dout::Float64)::Tuple{Float64,Float64} = (dout * layer.y, dout * layer.x)

backward! (generic function with 2 methods)

これで加算レイヤと乗算レイヤを実装できたため、これらを用いて少し複雑な計算グラフを実装する

前述した「りんご2個とみかん3個の買い物」の計算グラフを以下に示す

![computational_graph_backword.png](./img/computational_graph_backword.png)

これを実装すると以下のようになる

In [4]:
# 変数
apple = 100.0
apple_num = 2.0
orange = 150.0
orange_num = 3.0
tax = 1.1

# レイヤ（ノード）
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# 順伝播
apple_price = forward!(mul_apple_layer, apple, apple_num)
orange_price = forward!(mul_orange_layer, orange, orange_num)
all_price = forward!(add_apple_orange_layer, apple_price, orange_price)
price = forward!(mul_tax_layer, all_price, tax) # => 715

715.0000000000001

In [5]:
# 逆伝播
d_price = 1.0
d_all_price, d_tax = backward!(mul_tax_layer, d_price)
d_apple_price, d_orange_price = backward!(add_apple_orange_layer, d_all_price)
d_orange, d_orange_num = backward!(mul_orange_layer, d_orange_price)
d_apple, d_apple_num = backward!(mul_apple_layer, d_apple_price)

println("$d_apple_num, $d_apple, $d_orange_num, $d_orange, $d_tax")
# => d_apple_num: 110, d_apple: 2.2, d_orange_num: 165, d_orange: 3.3, d_tax: 650

110.00000000000001, 2.2, 165.0, 3.3000000000000003, 650.0


## 活性化関数レイヤの実装

計算グラフの考え方をニューラルネットワークに適用していく

### ReLUレイヤ
ReLU活性化関数は以下の式で表される

$$
    y = \begin{cases}
        x & (x > 0) \\
        0 & (x \leq 0)
    \end{cases}
$$

よって、微分は以下の式で表される

$$
    \frac{∂y}{∂x} = \begin{cases}
        1 & (x > 0) \\
        0 & (x \leq 0)
    \end{cases}
$$

従って、ReLU活性化関数を計算グラフで表すと以下のようになる

![computational_graph_relu.png](./img/computational_graph_relu.png)

In [6]:
# ReLUレイヤ
mutable struct ReluLayer <: AbstractLayer
    mask::AbstractArray{Bool}
    ReluLayer() = new()
end

# ReLUレイヤ: 順伝播
forward!(layer::ReluLayer, x::AbstractArray{Float64})::AbstractArray{Float64} = begin
    mask = layer.mask = (x .<= 0) # x <= 0 の要素をマスキング
    out = copy(x) # 入力値をそのまま出力値にコピー
    out[mask] .= zero(Float64) # x <= 0 でマスキングした要素それぞれに 0 代入
    out
end

# ReLUレイヤ: 逆伝播
backward!(layer::ReluLayer, dout::AbstractArray{Float64})::AbstractArray{Float64} = begin
    # 基本的に上流から流れてきた微分値をそのまま下流へ
    dout[layer.mask] .= zero(Float64) # x <= 0 でマスキングした要素それぞれに 0 代入
    dout
end

backward! (generic function with 3 methods)

In [7]:
"""
@test: ReLUレイヤ動作確認
"""
# ReLUレイヤ: 順伝播
## [1.0 -0.5; -2.0 3.0] => [1.0 0.0; 0.0 3.0]
relu = ReluLayer()
forward!(relu, [1.0 -0.5; -2.0 3.0])

2×2 Array{Float64,2}:
 1.0  0.0
 0.0  3.0

In [8]:
# ReLUレイヤ: マスク状態（x <= 0 のindex）
## => [false true; true false]
relu.mask

2×2 BitArray{2}:
 0  1
 1  0

In [9]:
# ReLUレイヤ: 逆伝播
## [1.0 1.0; 1.0 1.0] => [1.0 0.0; 0.0 1.0]
backward!(relu, [1.0 1.0; 1.0 1.0])

2×2 Array{Float64,2}:
 1.0  0.0
 0.0  1.0

### Sigmoidレイヤ
シグモイド関数は以下の式で表される

$$
    y = \frac{1}{1 + \exp(-x)}
$$

これを計算グラフで表すと以下のようになる

![computational_graph_sigmoid.png](./img/computational_graph_sigmoid.png)

この計算グラフの逆伝播をステップごとに分解して考えると以下のようになっている

1. 「／」ノードは $y=\frac{1}{x}$ を表し、この微分は解析的に以下のように解くことができる
    $$ \begin{array}{ll}
        \frac{∂y}{∂x} &= -\frac{1}{x^2} \\
                        &= -y^2
    \end{array} $$
2. 「＋」ノードは上流から流れてきた微分値をそのまま下流に流す（加算ノードの項参照）
3. 「exp」ノードは $y=e^x$ を表し、この微分は解析的に以下のように解くことができる
    $$
        \frac{∂y}{∂x} = e^x
    $$
4. 「×」ノードは、順伝播時の2値を "ひっくり返して" 乗算するため、ここでは -1 を乗算している

ここで、$\frac{∂L}{∂y}y^2 e^{-x}$ が順伝播の値 $x$, $y$ のみから計算可能であることに注目すると、上記計算グラフは、以下のように「sigmoid」ノードとしてグループ化することができる

![computational_graph_sigmoid2.png](./img/computational_graph_sigmoid2.png)

このようにグループ化することで途中の計算を省略することができるため、計算効率を向上させることができる

さらに $\frac{∂y}{∂x}y^2 e^{-x}$ は以下のように展開できるため、さらに整理できる

$$ \begin{array}{ll}
    \frac{∂y}{∂x}y^2 e^{-x} &= \frac{∂y}{∂x}\frac{1}{(1 + e^{-x})^2}e^{-x} & (∵ y = \frac{1}{1 + e^{-x}}) \\
    &= \frac{∂y}{∂x}\frac{1}{1 + e^{-x}}\frac{e^{-x}}{1 + e^{-x}} \\
    &= \frac{∂y}{∂x}y(1-y)
\end{array} $$

従って、最終的に計算グラフは以下のようにまとめることができる

![computational_graph_sigmoid3.png](./img/computational_graph_sigmoid3.png)

In [10]:
# Sigmoiodレイヤ
mutable struct SigmoidLayer <: AbstractLayer
    out::AbstractArray{Float64} # Sigmoidレイヤが保持しておかなければならないのは順伝播時の出力値 y
end

# Sigmoiodレイヤ: 順伝播
forward!(layer::SigmoidLayer, x::AbstractArray{Float64})::AbstractArray{Float64} =
    layer.out = 1 ./ (1 .+ exp.(-x))

# Sigmoiodレイヤ: 逆伝播
backward!(layer::SigmoidLayer, dout::AbstractArray{Float64})::AbstractArray{Float64} =
    dount .* layer.out .* (1 .- layer.out)

backward! (generic function with 4 methods)

計算グラフを使って途中計算を整理することによって、上記のように逆伝播（微分）を非常に簡単な計算に置き換えられるということは重要である

### Affineレイヤ
ニューラルネットワークの順伝播時で行われる計算は以下のようなものであった

$$
        出力行列 = 入力行列 \cdot 重み行列 + バイアス行列
$$

ここで、行列の内積計算は、幾何学分野で**アフィン変換**と呼ばれる

そのため、ニューラルネットワーク順伝播時の行列計算を行う層を**Affineレイヤ**として実装することにする

まず、計算グラフを考える

ここでは、バッチサイズを $N$, 入力値の数を 2, 重みを 2×3行列（入力数: 2, 出力数: 3）,  バイアスを 1×3行列（出力数: 3）として考えると、計算グラフは以下のようになる

![computational_graph_affine.png](./img/computational_graph_affine.png)

なお「dot」ノードは、実質的には「×」ノードと同一と考えて良いため、順伝播時の2入力値を "ひっくり返して" 乗算することで逆伝播の流れを作ることができる

ただし、行列の値をひっくり返すということは、転置行列を作ることを意味する

そのため、入力行列 $X$ の側における逆伝播は $\frac{∂L}{∂Y} \cdot W^T$ となり（$W^T$ は $W$ の転置行列）、重み行列 $W$ の側における逆伝播は $X^T \cdot \frac{∂L}{∂Y}$ となっている（$X^T$ は $X$ の転置行列）

In [14]:
# バッチ対応Affineレイヤ
mutable struct AffineLayer <: AbstractLayer
    # AbstractArray型にすることでコンストラクタですべてのメンバの値を指定を不要にする
    W::AbstractArray{Float64,2}
    B::AbstractArray{Float64,2}
    X::AbstractArray{Float64,2}
    dW::AbstractArray{Float64,2}
    dB::AbstractArray{Float64,2}
    
    # コンストラクタで重みとバイアスだけ指定できるように定義
    function AffineLayer(W::AbstractArray{Float64,2}, B::AbstractArray{Float64,2})
        layer = new()
        layer.W = W
        layer.B = B
        layer
    end
end

# Affineレイヤ: 順伝播
forward!(layer::AffineLayer, x::Array{Float64,2})::Array{Float64,2} = begin
    layer.X = x
    x * layer.W .+ layer.B
end

# Affineレイヤ: 逆伝播
backward!(layer::AffineLayer, dout::Array{Float64,2})::Array{Float64,2} = begin
    dX = dout * layer.W'
    layer.dW = layer.X' * dout
    layer.dB = sum(dout, dims=1) # バイアスの逆伝播値: ∂L/∂Y の最初の軸に対する和
    dX
end

backward! (generic function with 1 method)

In [15]:
"""
@test: Affineレイヤ動作確認
"""
# Affineレイヤ
## 重み行列: 1×3（入力数: 1, 出力数: 3）
## バイアス行列: 1×3（出力数: 3）
layer = AffineLayer([5.0 5.0 5.0], [1.0 2.0 3.0])

# 順伝播
## 入力行列: 2×1（バッチサイズ: 2, 特徴量の数: 1）
x = reshape([0.0; 2.0], 2, 1)
forward!(layer, x) # => 2×3 [1 2 3; 11 12 13]

2×3 Array{Float64,2}:
  1.0   2.0   3.0
 11.0  12.0  13.0

In [16]:
# 逆伝播
## 入力微分値: 2×3行列（バッチサイズ: 2, Affineレイヤの順伝播出力数: 3）
dY = [1.0 2.0 3.0; 4.0 5.0 6.0]
backward!(layer, dY) # => 2×1 行列 X に戻る [30; 75]

2×1 Array{Float64,2}:
 30.0
 75.0

In [17]:
# 逆伝播後の微分バイアス
layer.dB # => 1×3 [5 7 9]

1×3 Array{Float64,2}:
 5.0  7.0  9.0

### Softmaxレイヤ
Softmax活性化関数は以下の数式で表される

$$
    y_i = \frac{e^{x_i}}{\sum_{i=1}^N e^{x_i}}
$$

計算グラフで表すと以下のようになる

![computational_graph_softmax.png](./img/computational_graph_softmax.png)

続いて、①〜⑥の経路について逆伝播を考える

1. $\frac{∂L}{∂y_i}$
2. 「×」ノードのため、順伝播時の相方を乗算する
    $$
        \frac{∂L}{∂y_i} \cdot e^{x_i}
    $$
3. 「／」ノードのため、上流値を2乗し符号反転する
    - さらに各エッジの値を合計する
    $$
        \sum_{i=1}^{N} -(\frac{∂L}{∂y_i} \cdot e^{x_i})^2
    $$
4. 「×」ノードのため、順伝播時の相方を乗算する
    $$
        \frac{∂L}{∂y_i} \cdot \frac{1}{S} = \frac{∂L}{∂y_i} \cdot \frac{1}{\sum_{i=1}^{N}e^{x_i}}
    $$
5. 「＋」ノードのため、上流値をそのまま流す
    $$
        \sum_{i=1}^{N} -(\frac{∂L}{∂y_i} \cdot e^{x_i})^2
    $$
6. 「exp」ノードのため、順伝播時の出力値を乗算する
    $$
        e^{x_i} \cdot \sum_{i=1}^{N} -(\frac{∂L}{∂y_i} \cdot e^{x_i})^2
    $$